In [7]:
import os
import requests
import zipfile
import io

import numpy as np
import pandas as pd
import re
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [8]:
data_path_tsv = "data/mq_variants_intensity_cleaned.csv"

# May take a while to download
if not os.path.exists(data_path_tsv):
    os.makedirs("data", exist_ok=True)
    r = requests.post(
        "https://proteomics2.ucsd.edu/ProteoSAFe/DownloadResult?view=mq_variants_intensity&task=a19fe3be4bd84d4a80e58d64e14ba1dd",
        stream=True,
    )
    r.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        for zipinfo in z.infolist():
            if zipinfo.filename.endswith(".tsv"):
                zipinfo.filename = "mq_variants_intensity.tsv"
            z.extract(zipinfo, "data")

In [9]:
mq_variants_df = pd.read_csv(data_path_tsv)

C:\Users\17397\AppData\Local\Temp\ipykernel_9476\1702528689.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  mq_variants_df = pd.read_csv(data_path_tsv)


In [10]:
mq_variants_df.head()

,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,Charge,...,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod
0,1,.IITHPNFNGNTLDNDIM+15.995LIK.,37658,.IITHPNFNGNTLDNDIMLIK.,11683,20735,81,TRYP_PIG,2299.2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,.VADPDHDHTGFLTEYVATR.,93378,.VADPDHDHTGFLTEYVATR.,11372,15019,62,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,2144.0,2,...,182810000.0,182810000.0,296340000.0,296340000.0,272890000.0,272890000.0,254860000.0,254860000.0,70792000.0,70792000.0
2,3,.LGEHNIDVLEGNEQFINAAK.,50733,.LGEHNIDVLEGNEQFINAAK.,8878,23098,134,TRYP_PIG,2211.1,2,...,152910000.0,152910000.0,313690000.0,313690000.0,187600000.0,187600000.0,313290000.0,313290000.0,204790000.0,204790000.0
3,4,.FRHENIIGINDIIR.,25741,.FRHENIIGINDIIR.,8720,12619,33,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,1709.9,2,...,115160000.0,115160000.0,223460000.0,223460000.0,182890000.0,182890000.0,236530000.0,236530000.0,97725000.0,97725000.0
4,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,2,...,19220000.0,19220000.0,11216000.0,11216000.0,12721000.0,12721000.0,12835000.0,12835000.0,8137600.0,8137600.0


In [11]:
# Filter rows where Proteins column doesn't contain a semicolon
single_protein_variants = mq_variants_df[~mq_variants_df['Proteins'].str.contains(';', na=False)]

# Display the shape of the original and filtered dataframes
print(f"Original dataset shape: {mq_variants_df.shape}")
print(f"Filtered dataset shape: {single_protein_variants.shape}")

# Display first few rows of the filtered dataset
single_protein_variants.head()

Original dataset shape: (83706, 1032)
Filtered dataset shape: (16044, 1032)


,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,Charge,...,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod
0,1,.IITHPNFNGNTLDNDIM+15.995LIK.,37658,.IITHPNFNGNTLDNDIMLIK.,11683,20735,81,TRYP_PIG,2299.2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,.LGEHNIDVLEGNEQFINAAK.,50733,.LGEHNIDVLEGNEQFINAAK.,8878,23098,134,TRYP_PIG,2211.1,2,...,152910000.0,152910000.0,313690000.0,313690000.0,187600000.0,187600000.0,313290000.0,313290000.0,204790000.0,204790000.0
5,6,.NYLLSLPHK.,68115,.NYLLSLPHK.,7445,11842,41,sp|P28482|MK01_HUMAN,1084.6,2,...,290970000.0,290970000.0,477300000.0,477300000.0,363140000.0,363140000.0,43697000.0,43697000.0,182850000.0,182850000.0
11,12,.PM+15.995FIVNTNVPR.,69185,.PMFIVNTNVPR.,5232,10053,12,sp|P14174|MIF_HUMAN,1303.7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,.KLEAAEDIAYQLSR.,44634,.KLEAAEDIAYQLSR.,4958,6905,36,sp|P35232|PHB_HUMAN,1606.8,2,...,32624000.0,32624000.0,55130000.0,55130000.0,52332000.0,52332000.0,52156000.0,52156000.0,55709000.0,55709000.0


In [12]:
# Count rows with any NaN values
nan_rows = mq_variants_df.isna().any(axis=1).sum()
print(f"Number of rows with any NaN values: {nan_rows}")

# Calculate percentage
percentage = (nan_rows / len(mq_variants_df)) * 100
print(f"Percentage of rows with NaN values: {percentage:.2f}%")

# Show which columns have NaN values and their counts
nan_counts = mq_variants_df.isna().sum()
print("\nNumber of NaN values per column:")
print(nan_counts[nan_counts > 0])

Number of rows with any NaN values: 83706
Percentage of rows with NaN values: 100.00%

Number of NaN values per column:
All Mods                                              49397
Pep Prefix                                              286
Peptidoform                                           83706
Canonical proteins                                      863
Top canonical protein                                   863
                                                      ...  
_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod     76179
_dyn_#Baricitinib DMSO.Tech replicate 1 of 1          77883
_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod    75976
_dyn_#Baricitinib PDPD.Tech replicate 1 of 1          78812
_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod    77331
Length: 1006, dtype: int64


In [13]:
# Count rows with any NaN values
nan_rows = single_protein_variants.isna().any(axis=1).sum()
print(f"Number of rows with any NaN values: {nan_rows}")

# Calculate percentage
percentage = (nan_rows / len(single_protein_variants)) * 100
print(f"Percentage of rows with NaN values: {percentage:.2f}%")

# Show which columns have NaN values and their counts
nan_counts = single_protein_variants.isna().sum()
print("\nNumber of NaN values per column:")
print(nan_counts[nan_counts > 0])

Number of rows with any NaN values: 16044
Percentage of rows with NaN values: 100.00%

Number of NaN values per column:
All Mods                                               9343
Pep Prefix                                               42
Peptidoform                                           16044
Canonical proteins                                      413
Top canonical protein                                   413
                                                      ...  
_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod     14755
_dyn_#Baricitinib DMSO.Tech replicate 1 of 1          15065
_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod    14718
_dyn_#Baricitinib PDPD.Tech replicate 1 of 1          15235
_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod    14962
Length: 1006, dtype: int64


In [17]:
df_cleaned = single_protein_variants.dropna(how='all')
df_cleaned = df_cleaned.drop(columns=['Peptidoform'])

# Count rows with any NaN values
nan_rows = df_cleaned.isna().any(axis=1).sum()
print(f"Number of rows with any NaN values: {nan_rows}")

# Calculate percentage
percentage = (nan_rows / len(df_cleaned)) * 100
print(f"Percentage of rows with NaN values: {percentage:.2f}%")

# Show which columns have NaN values and their counts
nan_counts = df_cleaned.isna().sum()
print("\nNumber of NaN values per column:")
print(nan_counts[nan_counts > 0])

Number of rows with any NaN values: 16044
Percentage of rows with NaN values: 100.00%

Number of NaN values per column:
All Mods                                               9343
Pep Prefix                                               42
Canonical proteins                                      413
Top canonical protein                                   413
Unmod_Variant                                          6450
                                                      ...  
_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod     14755
_dyn_#Baricitinib DMSO.Tech replicate 1 of 1          15065
_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod    14718
_dyn_#Baricitinib PDPD.Tech replicate 1 of 1          15235
_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod    14962
Length: 1005, dtype: int64


In [ ]:
import re

# Identify columns with intensity data (excluding _unmod columns)
intensity_cols = [col for col in df_cleaned.columns if col.startswith('_dyn_#') and not col.endswith('_unmod')]

# Create a mapping of column to drug name
col_to_drug = {}
for col in intensity_cols:
    # Extract everything between '_dyn_#' and '.Tech'
    match = re.match(r'_dyn_#(.*?)\.Tech', col)
    if match:
        full_name = match.group(1)
        
        # Try to identify the concentration part
        conc_match = re.search(r' ([0-9]+nM|DMSO|PDPD)$', full_name)
        if conc_match:
            # Extract drug name by removing concentration
            drug = full_name[:conc_match.start()]
            col_to_drug[col] = drug
        else:
            # If no concentration is found, use the full name
            col_to_drug[col] = full_name

# Group columns by drug
drug_cols = {}
for col, drug in col_to_drug.items():
    if drug not in drug_cols:
        drug_cols[drug] = []
    drug_cols[drug].append(col)

# Create treatment-specific DataFrames
treatment_dfs = {}
for drug, cols in drug_cols.items():
    if len(cols) < 2:  # Skip drugs with only one concentration
        continue
        
    # Filter rows where all columns for this drug have non-NaN values
    mask = df_cleaned[cols].notna().all(axis=1)
    filtered_df = df_cleaned[mask].copy()
    
    if len(filtered_df) > 0:
        treatment_dfs[drug] = filtered_df.dropna(how='an')
        print(f"{drug}: {len(filtered_df)} peptides with complete intensity data for {len(cols)} concentrations")

print(f"\nCreated {len(treatment_dfs)} treatment-specific DataFrames")

AEE-788_inBT474: 683 peptides with complete intensity data for 10 concentrations
AEW-541: 289 peptides with complete intensity data for 10 concentrations
AMG-208: 356 peptides with complete intensity data for 10 concentrations
AMG-208_withCAKI: 246 peptides with complete intensity data for 10 concentrations
AMG-900: 2 peptides with complete intensity data for 10 concentrations
ARRY-380: 368 peptides with complete intensity data for 10 concentrations
ARRY-380_inBT474: 408 peptides with complete intensity data for 10 concentrations
ASP-3026: 272 peptides with complete intensity data for 10 concentrations
AT-13148: 219 peptides with complete intensity data for 10 concentrations
AT-7519: 282 peptides with complete intensity data for 10 concentrations
AT-9283: 264 peptides with complete intensity data for 10 concentrations
AV-412: 356 peptides with complete intensity data for 10 concentrations
AV-412_inBT474: 380 peptides with complete intensity data for 10 concentrations
AXL-1717: 333 pept

In [26]:
# Sample masked DF - 
treatment_dfs['AEW-541']["_dyn_#AEW-541 1000nM.Tech replicate 1 of 1"]

2        184990000.0
5        419250000.0
12        36813000.0
14       134780000.0
69        12136000.0
            ...     
24264      7819600.0
24770      6879500.0
27722       641850.0
31534      4436100.0
35139      3057200.0
Name: _dyn_#AEW-541 1000nM.Tech replicate 1 of 1, Length: 289, dtype: float64